In [ ]:
from dep_tools.azure import list_blob_container, get_container_client, download_blob
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [ ]:
client = get_container_client()
blobs = list(list_blob_container(client, "coastlines/0-4-11/mndwi-clean-lines/", ".gpkg"))

In [ ]:
# Shouldn't need to actually download them all, but that's what we're doing
# So this simple method is commented out...
# url = "https://deppcpublicstorage.blob.core.windows.net/output/coastlines/0-4-11/coastlines_merged_3857_0-4-11.gpkg.zip"
# dfs = [gpd.read_file(url + blob, engine="pyogrio") for blob in blobs]

for blob in blobs:
    out_file = Path(blob)
    if out_file.exists():
        continue
    out_folder = out_file.parent
    out_folder.mkdir(parents=True, exist_ok=True)
    with open(blob, "wb") as f:
        f.write(download_blob(client, blob))

In [ ]:
# This is the method for the in-memory version
# one_crs = [df.to_crs("epsg:3832") for df in dfs]

# This is for the on-disk version
one_crs = [gpd.read_file(blob, engine="pyogrio").to_crs("epsg:3857") for blob in blobs]

In [ ]:
# Merge all the geodataframes into one
merged = gpd.GeoDataFrame(pd.concat(one_crs, ignore_index=True))

In [ ]:
merged.to_file("coastlines_merged_3857.gpkg", driver="GPKG")

In [ ]:
from dep_tools.azure import copy_to_blob_storage

copy_to_blob_storage(client, "coastlines_merged_3857_0-4-11.gpkg.zip", "coastlines/0-4-11/coastlines_merged_3857_0-4-11.gpkg.zip")